<a href="https://colab.research.google.com/github/bhlr/Ciencia-de-Datos-en-Python-Seccion-U.../blob/main/proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El proyecto consiste en aplicar los conocimientos aprendidos en clase (y apoyándose de referencias adicionales útiles) para crear modelos predictivos de regresión lineal uni-variable sencillos de la forma:

y = f(x) =  mx +b

Donde:

y = la variable dependiente
x = variable independiente
m = pendiente de la recta(parámetro del modelo)
b = intercepto(parámetro del modelo)

Tanto “x” como “y” son parte del dataset usado , “m” y “b” son parámetros del modelo los cuales buscamos estimar con los datos, esto significa que se busca encontrar que valores de “m” y “b” producen una recta que describa de la mejor manera posible la relación entre los datos “x” y “y”.


In [4]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd

# Datos

Usando slicing como fue visto en clase el dataset usado será dividido de la siguiente forma:
 

*   80% del dataset(rows) se usará para todo el proceso(análisis exploratorio,selección de variables a usar, crear modelos predictivos). Les llamamos “datos de entrenamiento”
*   20% del dataset(rows) se usará para probar ,validar y evaluar los modelos resultantes.Esto significa que el 20% de data no es usado durante todo el proyecto y es usado solo al final del mismo cuando ya poseemos los modelos predictivos. Les llamamos “datos de validación y pruebas”


  


In [6]:
c=np.load("https://drive.google.com/file/d/18KJ-ZgNYR_EJEQUh0yMOJFHg7UGX1Z1E/view?usp=sharing")


FileNotFoundError: ignored